
<table align="center"><td>
  <a target="_blank"  href="https://github.com/ultralytics/yolov3/blob/master/tutorial.ipynb">
    <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on github
  </a>
</td><td>
  <a target="_blank"  href="https://colab.sandbox.google.com/github/ultralytics/yolov3/blob/master/tutorial.ipynb">
    <img width=32px src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
</td></table>

This notebook contains software developed by Ultralytics LLC, and **is freely available for redistribution under the GPL-3.0 license**. For more information please visit https://github.com/ultralytics/yolov3 and https://www.ultralytics.com.







In [0]:
import time
import glob
import torch
import os

from IPython.display import Image, clear_output 
print('PyTorch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Install NVIDIA Apex

In [0]:
%%writefile setup.sh

export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [0]:
!sh setup.sh

Load TensorBoard

In [0]:
!rm -rf /content/YoloPyTorch/runs/*

In [0]:
%load_ext tensorboard
%tensorboard --logdir /content/YoloPyTorch/runs

Mount Google Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')
# /content/drive/My Drive/Jewelry/images

---

*Clone* repository and download COCO 2014 dataset (20GB):

In [0]:
%cd /content
!git clone https://github.com/Goldenstarc/YoloPyTorch.git  # clone
# !git clone https://github.com/ultralytics/yolov3  # clone
!bash YoloPyTorch/data/get_coco2017.sh  # copy COCO2017 dataset (19GB)

Run `train.py` to train YOLOv3-Tiny starting from a darknet53 backbone:

In [18]:
%cd /content/YoloPyTorch
# !python3 train.py --cfg yolov3-spp-1cls.cfg --data data/jewelry1cls.data --weights '' --name ring1 --nosave --single-cls --cache-images --batch-size 8
!python3 train.py --cfg yolov3-tiny-7cls.cfg --data data/jewelry.data --weights '' --name ringAll --nosave --cache-images --batch-size 16

/content/YoloPyTorch
Namespace(adam=False, batch_size=16, bucket='', cache_images=True, cfg='yolov3-tiny-7cls.cfg', data='data/jewelry.data', device='', epochs=300, evolve=False, img_size=[320, 640], multi_scale=False, name='ringAll', nosave=True, notest=False, rect=False, resume=False, single_cls=False, weights='')
Using CUDA Apex device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
2020-05-07 20:49:23.365544: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Model Summary: 37 layers, 8.68374e+06 parameters, 8.68374e+06 gradients
Optimizer groups: 13 .bias, 13 Conv2d.weight, 11 other
Caching labels (806 found, 0 missing, 0 empty, 0 duplicate, for 806 images): 100% 806/806 [00:00<00:00, 6610.28it/s]
Caching images (1.0GB): 100% 806/806 [00:08<00:00, 96.13it/s]
Caching labels (806 found, 0 missing, 0 empty, 0 duplicate, 

Run `test.py` to evaluate the performance of a trained darknet or PyTorch model:

In [0]:
%cd /content/YoloPyTorch/

!python3 test.py --cfg yolov3-tiny-7cls.cfg --data data/jewelry.data  --weights weights/last_ringAll.pt --img 640 --augment --save-json

Run `detect.py` to perform inference on images in `data/samples` folder:

In [0]:
!python3 detect.py --cfg yolov3-tiny-7cls.cfg --weights weights/last_ringAll.pt --name data/jewelry.names --source 0
Image(filename='output/zidane.jpg', width=600)

Reproduce tutorial training runs and plot training results:

In [0]:
!python3 train.py --data data/coco_16img.data --batch-size 16 --accumulate 1 --nosave && mv results.txt results_coco_16img.txt  # CUSTOM TRAINING EXAMPLE
!python3 train.py --data data/coco_64img.data --batch-size 16 --accumulate 1 --nosave && mv results.txt results_coco_64img.txt 
!python3 -c "from utils import utils; utils.plot_results()"  # plot training results
Image(filename='results.png', width=800)

Extras below

---




In [0]:
!git pull

remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/Goldenstarc/YoloPyTorch
   0188cad..5d80078  master     -> origin/master
Updating 0188cad..5d80078
Fast-forward
 cfg/yolov3-tiny-7cls.cfg | 182 +++++++++++++++++++++++++++++++++++++++++++++++
 1 file changed, 182 insertions(+)
 create mode 100644 cfg/yolov3-tiny-7cls.cfg


In [0]:
# Unit Tests
!python3 detect.py --cfg yolov3-tiny-7cls.cfg  --weights weights/last_ringAll.pt --name data/jewelry.names

In [0]:
# Evolve Hyperparameters
!python3 train.py --data data/coco.data --img-size 320 --epochs 1 --evolve